# Create Dataframe Using Miocic et al. Data

This notebook contains the complete workthrough for creating the dataframe **miocic**, which is used primarily to [answer subquestion 1], and is expanded on to [answer sq 3]. This is a separate notebook to separate the data cleaning and data engineering methods used from the machine learning aspects.
# Import Necessary Packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Read in Miocic Data

In [4]:
miocic = pd.read_csv('miocic_database.csv')

In [5]:
miocic.head()

,Name,No,Depth_min,Depth_max,T_min,T_max,P_min,P_max,CO2_min,CO2_max,...,Reservoir,Reservoir thickness (Gross),Reservoir Thickness Net,Seal,Seal thickness,Stacked resevoirs,No. of stacked reservoirs,Literature,Well log & pressure,Offshore
0,Jackson Dome,1,4660.0,NaN,150.0,NaN,NaN,NaN,70.00,90.0,...,Carbonates; Sandstones,NaN,NaN,Mudstones,NaN,NaN,NaN,"Stevens et al., 2001; Stuart&Kosik, 1977; Rice...",NaN,No
1,El Trapial Field,10,1234.0,NaN,56.0,NaN,64.0,NaN,74.24,NaN,...,Sandstone,NaN,NaN,NaN,NaN,NaN,NaN,"Blann et al., 1997; Crotti et al, 2007; Orchue...",NaN,No
2,Messokampos,13,NaN,500,25.0,NaN,0.5,NaN,99.50,NaN,...,Sands,260,NaN,Clay,10s,NaN,NaN,"Gaus et al, 2004; TNO poster",NaN,No
3,Budafa Field,24,3200.0,3400,164.0,NaN,295.0,NaN,81.00,NaN,...,Sandstone,100,NaN,?,NaN,Yes,NaN,"Doleschall et al., 1992, Gacho-Muro, 2005 Kira...",NaN,No
4,DF1-1 Field,41,1280.0,1450,70.0,NaN,NaN,NaN,51.00,71.0,...,Sandstones,NaN,NaN,Mudstones,NaN,Yes,3,"Huang et al., 2003, 2004; Zhu et al, 2009, Zhe...",NaN,Yes


# Data Cleaning

In [6]:
miocic.columns

Index(['Name', 'No', 'Depth_min', 'Depth_max', 'T_min', 'T_max', 'P_min ',
       'P_max', 'CO2_min', 'CO2_max', 'Densities', 'Leakage',
       'Leakage pathways', 'Basin Type', 'CO2 source', 'CO2 held for',
       'Size (Tcf)', 'Trapping mechanism', 'Fault bound/sealed?', 'Reservoir',
       'Reservoir thickness (Gross)', 'Reservoir Thickness Net', 'Seal',
       'Seal thickness', 'Stacked resevoirs', 'No. of stacked reservoirs',
       'Literature', 'Well log & pressure ', 'Offshore'],
      dtype='object')

In [7]:
miocic.describe()

,No,Depth_min,T_min,T_max,P_min,CO2_min,CO2_max,Densities,Basin Type,Reservoir Thickness Net
count,61.000000,60.000000,58.000000,18.000000,31.000000,52.000000,25.000000,61.000000,0.0,5.000000
mean,31.000000,2066.195000,85.177534,87.976667,225.774194,82.631731,83.228000,504.273770,NaN,26.800000
std,17.752934,1035.096305,36.940722,38.054862,162.733135,17.850647,17.489813,219.883799,NaN,14.446453
min,1.000000,375.000000,20.000000,32.920000,0.500000,20.000000,42.900000,0.900000,NaN,12.000000
25%,16.000000,1268.500000,56.375000,59.750000,77.900000,71.587500,77.000000,381.400000,NaN,21.000000
50%,31.000000,2012.000000,85.500000,87.142500,209.000000,90.000000,90.000000,549.000000,NaN,21.000000
75%,46.000000,2764.000000,100.300000,104.625000,292.500000,97.000000,97.050000,656.900000,NaN,30.000000
max,61.000000,4660.000000,200.000000,165.000000,620.000000,99.700000,99.500000,919.300000,NaN,50.000000


In [8]:
miocic.set_index('Name', inplace=True)

## Drop Columns

- 'No' is used to number the basins and is therefore not relevant to the model.
- 'CO2 min' describes the minimum amount of CO2 stored. Since this data isn't an intrinsic part of potential sites for carbon injection, it is not relevant
- 'CO2 max' describes the minimum amount of CO2 stored. Since this data isn't an intrinsic part of potential sites for carbon injection, it is not relevant. 
- 'N2 max' describes amounts of other gases in the basin. A majority of basins do not have this information, so it is dropped.
- 'H2S max' describes the amounts of other gases in the basin. A majority of basins do not have this information, so it is dropped.
- 'C1 max' only has data for a few basins, so it is dropped.
- 'Leakage pathways' has data only for insecure basins, so it is dropped.
- 'Basin type' does not have any data, so it is dropped.
- 'CO2 source' is not relevant to potential sites for carbon injection and is therefore not relevant to the model.
- 'CO2 held for' is not relevant to potential sites for carbon injection and is therefore not relevant to the model.
- 'Size (Tcf)' has limited data availability, so it is dropped.
- 'Reservoir Thickness Net' has limited data availability so it is dropped.
- 'No. of stacked reservoirs' has limited data availability, so it is dropped.
- 'Literature' cites the article with the basin information, so it is not relevant to the model.
- 'Well log & pressure' demonstrates if there is well log data information so it is not relevant to the model.

In [9]:
#drop columns
miocic.drop(['No', 'CO2_min', 'CO2_max', 'Leakage pathways', 
            'Basin Type', 'CO2 source', 'CO2 held for', 'Size (Tcf)',
            'No. of stacked reservoirs', 'Well log & pressure ', 'Literature', 
             'Reservoir Thickness Net'], axis=1, inplace=True)

## Convert columns to necessary datatype

In [10]:
miocic.dtypes

Depth_min                      float64
Depth_max                       object
T_min                          float64
T_max                          float64
P_min                          float64
P_max                           object
Densities                      float64
Leakage                         object
Trapping mechanism              object
Fault bound/sealed?             object
Reservoir                       object
Reservoir thickness (Gross)     object
Seal                            object
Seal thickness                  object
Stacked resevoirs               object
Offshore                        object
dtype: object

In [11]:
def str_to_float(col):
    miocic[col] = pd.to_numeric(
        miocic[col]
        .str.strip() # remove extra spaces
        .replace(r'[^0-9.-]', '', regex=True), # turn "NA" into NaN
        errors= 'coerce'
    )
for col in ['Depth_max', 'P_max', 'Reservoir thickness (Gross)', 'Seal thickness']:
    str_to_float(col)

## Agregate min/max columns to average

- Depth
- Temperature
- Pressure

In [12]:
#depth
miocic['Depth'] = miocic[['Depth_min', 'Depth_max']].mean(axis=1, skipna=True)
#temperature
miocic['Temperature'] = miocic[['T_min', 'T_max']].mean(axis=1, skipna=True)
#pressure
miocic['Pressure'] = miocic[['P_min ', 'P_max']].mean(axis=1, skipna=True)
#drop min/max columns
miocic.drop(['Depth_min', 'Depth_max', 'T_min', 'T_max',
            'P_min ', 'P_max'], axis=1, inplace=True)

In [13]:
miocic['Depth'].isna().any()

False

In [14]:
sum(miocic['Temperature'].isna())

3

In [15]:
sum(miocic['Pressure'].isna())

30

After aggregating min and max columns, we are still missing a lot of pressure data.

In [16]:
miocic.head()

,Densities,Leakage,Trapping mechanism,Fault bound/sealed?,Reservoir,Reservoir thickness (Gross),Seal,Seal thickness,Stacked resevoirs,Offshore,Depth,Temperature,Pressure
Name,,,,,,,,,,,,,
Jackson Dome,654.6,No,Structural and salt structures,NaN,Carbonates; Sandstones,NaN,Mudstones,NaN,NaN,No,4660.0,150.0,NaN
El Trapial Field,141.0,No,"Dome, Structural",NaN,Sandstone,NaN,NaN,NaN,NaN,No,1234.0,56.0,64.0
Messokampos,0.9,Yes,Stratigraphic,NaN,Sands,260.0,Clay,10.0,NaN,No,500.0,25.0,0.5
Budafa Field,451.6,No,NaN,NaN,Sandstone,100.0,?,NaN,Yes,No,3300.0,164.0,295.0
DF1-1 Field,378.3,No,Mudstone Diapir,NaN,Sandstones,NaN,Mudstones,NaN,Yes,Yes,1365.0,70.0,NaN


# Deal with categorical variables

## 'Leakage' Feature

In [17]:
#Leakage has 3 options: Yes, No, and Inconclusive. 
#I will deal with the inconclusive sites later, but for now I want to use integers to represent this data
miocic['Leakage']= miocic['Leakage'].str.strip().map(
    {'No': 0, 'Yes': 1, 'Inconclusive': 2}).astype('int')

## 'Trapping mechanism' Feature

In [18]:
miocic['Trapping mechanism'].value_counts()

Trapping mechanism
Structural (fault)                             4
structural, stratigraphic                      4
Mudstone Diapir                                3
Stratigraphic                                  3
Antiform                                       3
Anticline                                      3
Diapir, fault                                  2
Structural                                     2
Structural and salt structures                 1
Dome, Structural                               1
Horst (structural)                             1
Dome Anticline                                 1
Structural stratigraphic                       1
Structural (4 way closure)                     1
Dome                                           1
Anticline, faulted                             1
Anticlinal fold with thrust fault              1
anticline, faulted                             1
Structural/stratigraphic (intrusion bound)     1
Stratigraphic                                  1
S

In [19]:
miocic['Trapping mechanism'].str.findall(r'[Ss]tructural')

Name
Jackson Dome                [Structural]
El Trapial Field            [Structural]
Messokampos                           []
Budafa Field                         NaN
DF1-1 Field                           []
                                ...     
Turaco, Albertine Graben             NaN
DF29-1                                []
L22-1                                 []
Kalangadoo 1                         NaN
W961                                 NaN
Name: Trapping mechanism, Length: 61, dtype: object

In [20]:
miocic['Trapping mechanism'].str.contains(r'[Ff]ault').value_counts()

Trapping mechanism
False    35
True     17
Name: count, dtype: int64

I will not be employing traditional one-hot encoding for 'Trapping Mechanism'. Instead, I have identified key trapping mechanisms and I will use regular expressions to identify if a basin has those mechanisms. This deals with basins having more than one trap identified. 

<u>Trap mechanisms</u>  
- Structural  
- Fault  
- Stratigraphic  
- Anticline (including antiform)  
- Horst  
- Lithological  
- Dome  
- Bound  
- Diapir (including salt structures)

In [21]:
#structural
miocic['Trapmech_structural'] = miocic['Trapping mechanism'].str.contains(
    r'[Ss]tructural|Dome$') #24 basins in this category
#fault
miocic['Trapmech_fault'] = miocic['Trapping mechanism'].str.contains(
    r'[Ff]ault') #17 basins in this category
#stratigraphic
miocic['Trapmech_stratigraphic'] = miocic['Trapping mechanism'].str.contains(
    r'[Ss]tratigraphic') #12 basins in this category
#anticline (including antiform)
miocic['Trapmech_anticline'] = miocic['Trapping mechanism'].str.contains(
    r'[Aa]nticline|[Aa]ntiform') #14 basins in this category
#horst
miocic['Trapmech_horst'] =miocic['Trapping mechanism'].str.contains(
    r'[Hh]orst') # 2 basins in this category
#lithological
miocic['Trapmech_lithological'] = miocic['Trapping mechanism'].str.contains(
    r'[Ll]ithological', na=False) # 2 basins in this category
#dome
miocic['Trapmech_dome'] = miocic['Trapping mechanism'].str.contains(
    r'[Dd]ome') # 3 basins in this category
#bound
miocic['Trapmech_bound'] = miocic['Trapping mechanism'].str.contains(
    r'[Bb]ound') #6 basisns in this category
#diapir (including salt structures)
miocic['Trapmech_diapir'] = miocic['Trapping mechanism'].str.contains(
    r'[Dd]iapir|[Ss]alt') #6 basins in this category

In [22]:
#drop 'Trapping mechanism' column
#miocic.drop(['Trapping mechanism'], axis=1, inplace=True)

## 'Fault bound/sealed' Feature

For this categorical variable, the data can either be 'yes', 'no', or 'na', which is different from missing a value. I will one-hot encode this variable but I also have to differentiate between 'na' and missing values, since the computer does not differentiate this.

In [23]:
miocic['Fault bound/sealed?'].value_counts()

Fault bound/sealed?
Yes    30
No     14
yes     4
no      3
Name: count, dtype: int64

Currently, this variable is stored as strings and NaN values. I need to standardize the capitalization  before one-hot encoding.

In [24]:
miocic['Fault bound/sealed?'] = miocic['Fault bound/sealed?'].str.lower()
miocic['Fault bound/sealed?'].value_counts()

Fault bound/sealed?
yes    34
no     17
Name: count, dtype: int64

I am using OneHotEncoder from sklearn to one-hot encode this categorical variable.

In [25]:
fault = OneHotEncoder()
fault.fit(miocic[['Fault bound/sealed?']])
fault_columns = pd.DataFrame(fault.transform(miocic[['Fault bound/sealed?']]).toarray(), 
                                  columns=fault.get_feature_names_out(), index=miocic.index)
fault_columns.rename(columns={'Fault bound/sealed?_no': 'Faultbound_no',
                              'Fault bound/sealed?_yes': 'Faultbound_yes',
                              'Fault bound/sealed?_nan': 'Faultbound_nan'}, inplace=True)
#the last two basins have missing data for this variable, so I am manually changing this
fault_columns.iloc[59:, :] = np.nan
#remove a column to maintain linear independence
fault_columns.drop('Faultbound_no', axis=1, inplace=True)

Now that I have created fault_columns, a dataframe of the one-hot encoded feature, I merge this dataframe with the miocic dataframe.

In [26]:
#combine one hot encoded df to miocic
miocic= pd.merge(miocic, fault_columns, left_on='Name', right_on='Name')

## 'Reservoir' Feature

In [27]:
miocic.shape

(61, 24)

In [28]:
len(miocic['Reservoir'].value_counts().index)
#aviso: Basement, carbonate, dolomite, limestone, sandstone, mixed, sands, volcanics

28

In [29]:
miocic['Reservoir'].value_counts()

Reservoir
Sandstone                                      17
Sandstones                                      7
Carbonates                                      5
Carbonate                                       4
Dolomite                                        2
Carbonates; Sandstones                          2
Sandstone, siltstone                            2
Limestone                                       2
Sandstone, Siltstone, Limestone & Anhydrite     1
Sandstone & Carbonates                          1
Sands                                           1
Limestone; Dolomite                             1
Volcanics, conglomerates                        1
Dolomite, Limestone                             1
Siltstones; Carbonates                          1
Sandstones                                      1
Sandstones; Mudstones                           1
Sandstone & Limestone                           1
Limestones                                      1
Sandstone, Limestone                    

In [30]:
#aviso: Basement, carbonate, dolomite, limestone, sandstone, mixed, sands, volcanics
#my categories: other (basement, sands), sandstones/conglomerates, siltstones/mudstones, carbonites (ites)
miocic['Reservoir'].str.contains(r'Sandstones?|[Mm]udstones?').value_counts()


Reservoir
True     35
False    26
Name: count, dtype: int64

In [31]:
miocic['Reservoir'].str.contains(r'[Ll]imestone').value_counts()

Reservoir
False    51
True     10
Name: count, dtype: int64

In [32]:
miocic['Reservoir'].str.findall(r'[Ss]iltstones?|[Mm]udstones?').value_counts()


Reservoir
[]              55
[Siltstone]      2
[siltstone]      2
[Siltstones]     1
[Mudstones]      1
Name: count, dtype: int64

Instead of traditional one-hot encoding, which would not be successful due to the uniqueness/fluidity of categories, I will employ heirarchical categorization, essentially creating a high level categorization and a detailed categorizatinon. For the high level categorization, I will sort all the basin reservoir types into 4 options:

<u>Level 1 Reservoir Type</u> 
- Sandstones/clastics
    - conglomerates
    - siltstones
    - mudstones
    - sands
- Carbonates
    - limestone
    - dolomites
    - biolithite
- Mixed Lithology
- Other
    - Volcanics
    - Basement

Then, I will one-hot encode this high level reservoir type feature.

However, to not loose information, I will then be doing a second-level categorization using regular expressions to feature engineer if certain key categories are listed. This way, I can highlight subcategories and the 'Mixed' basisns can be labeled as multiple categories. Doing this, a single basin can belong to multiple categories.

<u>Level 2 Reservoir Type</u>  
- Sandstones (includes conglomerates)
- Sands
- Siltstones/mudstones
- Carbonates (includes biolithite)
- Dolomites
- Limestones
- Anhydrites/evaporites
- Volcanics
- Basement

In [33]:
miocic['L1reservoir'] = miocic['Reservoir'].str.strip().map({
    # Sandstones / clastics
    'Sandstone': 'Sandstone',
    'Sandstones': 'Sandstone',
    'Sandstone, siltstone': 'Sandstone',
    'Sands': 'Sandstone',
    'Sandstones; Mudstones': 'Sandstone',
    'Siltstone, sandstone': 'Sandstone',
    'Conglomerates, Sandstones': 'Sandstone',

    # Carbonates
    'Carbonates': 'Carbonate',
    'Carbonate': 'Carbonate',
    'Dolomite': 'Carbonate',
    'Limestone': 'Carbonate',
    'Limestones': 'Carbonate',
    'Limestone; Dolomite': 'Carbonate',
    'Dolomite, Limestone': 'Carbonate',
    'Dolomites, Evaporites': 'Carbonate',
    'Biolithite, dolomite': 'Carbonate',

    # Mixed lithologies
    'Carbonates; Sandstones': 'Mixed_lith',
    'Sandstone, Siltstone, Limestone & Anhydrite': 'Mixed_lith',
    'Sandstone & Carbonates': 'Mixed_lith',
    'Sandstone & Limestone': 'Mixed_lith',
    'Sandstone, Limestone': 'Mixed_lith',
    'Dolomitic limestone, sandstone': 'Mixed_lith',
    'Siltstones; Carbonates': 'Mixed_lith',

    # Other / rare
    'Volcanics': 'Other',
    'Volcanics, conglomerates': 'Other',
    'Basement': 'Other'
})


In [34]:
miocic[['L1reservoir']].isna().any() #checking that all were re-categorized

L1reservoir    False
dtype: bool

In [35]:
#one-hot encode L1reservoir
resv = OneHotEncoder()
resv.fit(miocic[['L1reservoir']])
resv_columns = pd.DataFrame(resv.transform(miocic[['L1reservoir']]).toarray(), 
                                  columns=resv.get_feature_names_out(), index=miocic.index)
resv_columns.head()
#in order to maintain linear independce, I need to drop a row. No data is lost when doing this.
resv_columns.drop('L1reservoir_Other', axis=1, inplace=True)
resv_columns.head()
#merge with miocic
miocic = pd.merge(miocic, resv_columns, left_on= 'Name', right_on='Name')
miocic.head()

,Densities,Leakage,Trapping mechanism,Fault bound/sealed?,Reservoir,Reservoir thickness (Gross),Seal,Seal thickness,Stacked resevoirs,Offshore,...,Trapmech_lithological,Trapmech_dome,Trapmech_bound,Trapmech_diapir,Faultbound_yes,Faultbound_nan,L1reservoir,L1reservoir_Carbonate,L1reservoir_Mixed_lith,L1reservoir_Sandstone
Name,,,,,,,,,,,,,,,,,,,,,
Jackson Dome,654.6,0,Structural and salt structures,NaN,Carbonates; Sandstones,NaN,Mudstones,NaN,NaN,No,...,False,False,False,True,0.0,1.0,Mixed_lith,0.0,1.0,0.0
El Trapial Field,141.0,0,"Dome, Structural",NaN,Sandstone,NaN,NaN,NaN,NaN,No,...,False,True,False,False,0.0,1.0,Sandstone,0.0,0.0,1.0
Messokampos,0.9,1,Stratigraphic,NaN,Sands,260.0,Clay,10.0,NaN,No,...,False,False,False,False,0.0,1.0,Sandstone,0.0,0.0,1.0
Budafa Field,451.6,0,NaN,NaN,Sandstone,100.0,?,NaN,Yes,No,...,False,NaN,NaN,NaN,0.0,1.0,Sandstone,0.0,0.0,1.0
DF1-1 Field,378.3,0,Mudstone Diapir,NaN,Sandstones,NaN,Mudstones,NaN,Yes,Yes,...,False,False,False,True,0.0,1.0,Sandstone,0.0,0.0,1.0


In [36]:
#L2reservoir feature engineering

#sandstones (including conglomerates)
miocic['L2reservoir_sandstone'] = miocic['Reservoir'].str.contains(
    r'[Ss]andstones?|[Cc]onglomerates?') #38 basins in this category
#sands
miocic['L2reservoir_sands'] = miocic['Reservoir'].str.strip().str.contains(
    r'[Ss]ands?$') #1 basin
#siltstone (including mudstone)
miocic['L2reservoir_siltstone']= miocic['Reservoir'].str.contains(
    r'[Mm]udstones?|[Ss]iltstones?') #6 basins
#carbonates (including biolithite)
miocic['L2reservoir_carbonate']= miocic['Reservoir'].str.contains(
    r'[Cc]arbonates?|Biolithite') #14 basins
#dolomites
miocic['L2reservoir_dolomite'] = miocic['Reservoir'].str.contains(
    r'[Dd]olomit') #7 basins
#limestones 
miocic['L2reservoir_limestone'] = miocic['Reservoir'].str.contains(
    r'[Ll]imestone') #10 basins
#Evaporites (including anhydrites)
miocic['L2reservoir_evaporite'] = miocic['Reservoir'].str.contains(
    r'Anhydrite|Evaporite') # 2 basins
#Volcanics
miocic['L2reservoir_volcanic'] = miocic['Reservoir'].str.contains(
    r'[Vv]olcanics?') # 2 basins
#Basement
miocic['L2reservoir_basement'] = miocic['Reservoir'].str.contains(
    r'Basement') #1 basin

## Thickness

In [37]:
sum(miocic['Reservoir thickness (Gross)'].isna())

18

There are 18 missing values. This is roughly 30% of the data.

In [38]:
sum(miocic['Seal'].isna())

6

In [39]:
miocic['Seal'].value_counts()

Seal
Mudstone                                    10
Shale                                        9
Mudstones                                    7
Clay                                         1
Evaporites; Shales                           1
Salt                                         1
?                                            1
Claystone/Siltstone                          1
Anhydrite & claystones                       1
Mudstones; Anhydrite                         1
Thrust deposits                              1
Evaporite, Carbonates                        1
Shale/Clay                                   1
Shale and Siltstones                         1
Halite                                       1
Anhydrite; Mudstones                         1
Marine sediments (mudstones?); Laccolith     1
Shale & Limestone                            1
Evaporites; Mudstones                        1
Shale                                        1
Claystone and Marlstone                      1
Shale, E

In [51]:
sum(miocic['Reservoir thickness (Gross)'].isna())

18

In [45]:
miocic.iloc[0, :]

Densities                                               654.6
Leakage                                                     0
Trapping mechanism             Structural and salt structures
Fault bound/sealed?                                       NaN
Reservoir                              Carbonates; Sandstones
Reservoir thickness (Gross)                               NaN
Seal                                                Mudstones
Seal thickness                                            NaN
Stacked resevoirs                                         NaN
Offshore                                                   No
Depth                                                  4660.0
Temperature                                             150.0
Pressure                                                  NaN
Trapmech_structural                                      True
Trapmech_fault                                          False
Trapmech_stratigraphic                                  False
Trapmech

In [53]:
miocic[miocic['Leakage']== 1]

,Densities,Leakage,Trapping mechanism,Fault bound/sealed?,Reservoir,Reservoir thickness (Gross),Seal,Seal thickness,Stacked resevoirs,Offshore,...,L1reservoir_Sandstone,L2reservoir_sandstone,L2reservoir_sands,L2reservoir_siltstone,L2reservoir_carbonate,L2reservoir_dolomite,L2reservoir_limestone,L2reservoir_evaporite,L2reservoir_volcanic,L2reservoir_basement
Name,,,,,,,,,,,,,,,,,,,,,
Messokampos,0.9,1,Stratigraphic,NaN,Sands,260.0,Clay,10.0,NaN,No,...,1.0,False,True,False,False,False,False,False,False,False
St. Johns Dome,182.3,1,"Anticline, faulted",yes,Siltstones; Carbonates,200.0,Anhydrite; Mudstones,300.0,Yes,No,...,0.0,False,False,True,True,False,False,False,False,False
Farnham Dome,194.7,1,"anticline, faulted",yes,Sandstone,100.0,Shale & Limestone,100.0,No,No,...,1.0,True,False,False,False,False,False,False,False,False
Latera caldera,119.3,1,Structural high,yes,Limestones,NaN,Flysch and Volcanics,NaN,NaN,No,...,0.0,False,False,False,False,False,True,False,False,False
Pieve Santo Stefano,830.4,1,Structural/straticgraphic,yes,"Dolomites, Evaporites",240.0,Evaporites (?),300.0,No,No,...,0.0,False,False,False,False,True,False,True,False,False
Frigento Field,199.7,1,Antiform,yes,Carbonates,427.0,Mudstones,150.0,No,No,...,0.0,False,False,False,True,False,False,False,False,False
